##### Imports

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.feature_selection \
        import mutual_info_regression
        
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [3]:
from pathlib import Path
import pandas as pd
from filelock import FileLock

# Define the path
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')
lock_path = "/tmp/assets_h5_file.lock"  # Lock path

# Lock the file and retrieve the data
with FileLock(lock_path):
    with pd.HDFStore(DATA_STORE) as store:
        alpha_101_data = store['factors/alpha_101']
        common_data = store['factors/common']

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1652, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 632, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048

End of HDF5 error back trace

Unable to open/create file '/home/sayem/Desktop/Project/data/assets.h5'

In [ ]:

# Remove overlapping column from common_data
common_data = common_data.drop(columns=['ret_fwd_01d'])

# Join the dataframes
joined_data = common_data.join(alpha_101_data, \
    how='right', lsuffix='_alpha', rsuffix='_common')


In [ ]:
CHUNK_SIZE = 10000  # Adjust based on available memory
total_chunks = len(joined_data) // CHUNK_SIZE + 1

for i in range(total_chunks):
    start_idx = i * CHUNK_SIZE
    end_idx = (i + 1) * CHUNK_SIZE
    chunk = joined_data.iloc[start_idx:end_idx]
    
    chunk.to_hdf(DATA_STORE, 'factors/concat', format='table', mode='a', \
        append=True, complevel=9, complib='blosc')
    print(f"Saved chunk {i+1} of {total_chunks}")

# Free up memory
del joined_data


In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    concat = store['factors/concat']